In [ ]:
import os
import cv2
import glob
import ntpath
import math
import numpy
import pydicom  # pip install pydicom

In [ ]:
extract_dicom_images()

In [ ]:
def extract_dicom_images():
    
    src_path = "G:/LungCancerPredict/original/ndsb_raw/"
    src_files = glob.glob(src_path + "*")
    
    count=1
    
    for src_file in src_files:
         
        print("Counter:",count)
        
        extract_dicom_image(src_file)
        
        count+=1

def extract_dicom_image(src_file):
    
    patient_id = ntpath.basename(src_file)
    
    #创建存储路径
    dst_path = "G:/LungCancerPredict/extracted/ndsb3_extracted_images/" + patient_id + "/"
    os.makedirs(dst_path, exist_ok=True)
    
    #读取数据
    slices = load_slices(src_file)
    img_array = convert_pixel_to_hu(slices)
    print("Original Shape:",img_array.shape)
    
    #如果图片有倾斜旋转，之后需要处理
    cos_value = (slices[0].ImageOrientationPatient[0])
    cos_degree = round(math.degrees(math.acos(cos_value)),2)
    
    #如果图片是倒叙的，需要调整顺序
    invert_order = slices[1].ImagePositionPatient[2] > slices[0].ImagePositionPatient[2]
    if not invert_order: img_array = numpy.flipud(img_array)

    #重采样
    spacing = slices[0].PixelSpacing
    spacing.append(slices[0].SliceThickness)
    img_array = rescale_image(img_array, spacing)
    print("Rescale Shape:",image.shape)

    for i in range(img_array.shape[0]):
        
        img = img_array[i]
        
        #用来处理图像的倾斜旋转
        if cos_degree>0.0: img = cv_flip(img, img.shape[1], img.shape[0], cos_degree)
        
        #肺分割
        seg_img, mask = get_segmented_lungs(img.copy())
        
        #数据标准化
        img = normalize_hu(img)
        
        dst_file = dst_path + "img_" + str(i).rjust(4, '0') + "_i.png"
        cv2.imwrite(dst_file, img * 255)
        cv2.imwrite(dst_file.replace("_i.png", "_m.png"), mask * 255)

def load_slices(src_dir):
    
    slices = [pydicom.read_file(src_dir + '/' + s) for s in os.listdir(src_dir)]
    slices.sort(key=lambda slice: int(slice.InstanceNumber))

    try:
        slice_thickness = numpy.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = numpy.abs(slices[0].SliceLocation - slices[1].SliceLocation)

    for s in slices:
        
        s.SliceThickness = slice_thickness
        
    return slices

def convert_pixel_to_hu(slices):
    
    img = numpy.stack([s.pixel_array for s in slices])
    img = img.astype(numpy.int16)
    img[img == -2000] = 0
    
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        img[slice_number] = slope * img[slice_number] + intercept

    return numpy.array(img, dtype=numpy.int16)

def rescale_image(images_zyx, org_spacing_xyz):
    
    target_voxel_mm = 1.00
    resize_z = float(org_spacing_xyz[2]) / float(target_voxel_mm)
    res = cv2.resize(images_zyx, dsize=None, fx=1.0, fy=resize_z, interpolation=cv2.INTER_LINEAR)  # cv2.resize默认数据按照y,x,channel排列
    
    res = res.swapaxes(0, 2)
    res = res.swapaxes(0, 1)
 
    resize_x = float(org_spacing_xyz[0]) / float(target_voxel_mm)
    resize_y = float(org_spacing_xyz[1]) / float(target_voxel_mm)
    res = cv2.resize(res, dsize=None, fx=resize_x, fy=resize_y, interpolation=cv2.INTER_LINEAR)

    res = res.swapaxes(0, 2)
    res = res.swapaxes(2, 1)
   
    return res

def get_segmented_lungs(img):
    
    binary = img < -400                                      # step1: convert to binary
    
    cleared = clear_border(binary)                           # step2: remove blobs connected to the border

    label_image = label(cleared)                             # step3: label the image
    
    areas = [r.area for r in regionprops(label_image)]       # step4: keep the labels with 2 largest areas
    areas.sort()

    if len(areas) > 2:
        
        for region in regionprops(label_image):
            
            if region.area < areas[-2]:
            
                for coordinates in region.coords:
                    
                       label_image[coordinates[0], coordinates[1]] = 0
                
    binary = label_image
    
    kernel = disk(2)                                         # step5: erosion operation with a disk of radius 2
    binary = binary_erosion(binary, kernel)
    
    kernel = disk(10)                                        # step6: closure operation with a disk of radius 10
    binary = binary_closing(binary, kernel)                   
    
    edges = roberts(binary)                                  # step7: fill in the small holes inside the binary mask of lungs
    binary = ndi.binary_fill_holes(edges)
    
    get_high_vals = binary == 0                              # step8: superimpose the binary mask on the input image
    img[get_high_vals] = -2000
    
    return img, binary

def normalize_hu(img):
    
    MIN_BOUND = -1000.0
    MAX_BOUND = 400.0
    
    img = (img - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    
    img[img > 1] = 1
    img[img < 0] = 0
    
    return img